# Decision Making LCA Tool

Welcome to the program! This program will help you simulate LCA scenarios for both Traditional Manufacturing and Additive Manufacturing and enable you to compare them

In [1]:
#### Importing necessary libraries

# Libraries for calculations
import numpy as np

from datetime import datetime as date
import functools

# Libraries for displaying information
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive,interact, HBox, Layout,VBox
from IPython.display import display

In [2]:
date.now().strftime("%d-%m-%Y-%H:%M:%S")

'06-02-2023-14:36:47'

## Defining useful functions

In [3]:
def MaterialProduction(EmbodiedMat1, EmbodiedMat2, massTM, massAM):
    '''
    Description:
    ------------

    This function will calculate material production energy requirements and 
    CO2 emissions for both TM and AM simultaneously by multiplying the array 
    of embodied energy and carbon by the mass relevant for the given path
    
    Variables:
    ----------
    
    EmbodiedMat1: 
    - Type: Array
    - Desc: Array with the embodied energy (MJ/kg) and carbon (kgCO2/kg) of the 
    material used in TM,  - [Energy, Carbon].
    
    EmbodiedMat2: 
    - Type: Array
    - Desc: Array with the embodied energy (MJ/kg) and carbon (kgCO2/kg) of the 
    material used in AM,  - [Energy, Carbon].
    
    massTM: 
    - Type: Float
    - Desc: Stores the mass of the initial workpiece that will be used in TM path.
    
    massAM: 
    - Type: Float
    - Desc: Stores the mass of the mass that will be placed on the AM path (has 
    to consider support mass and allowance of the machine)
    
    Returns:
    --------
    
    EG_TM:
    - Type: Array
    - Desc: Stores the energy and carbon demand for the material production for 
    the workpiece in TM
    
    EG_AM:
    - Type: Array
    - Desc: Stores the energy and carbon demand for the material production of 
    the material that will be placed in AM
    
    
    '''
    
    EG_TM = np.multiply(EmbodiedMat1,massTM)
    EG_AM = np.multiply(EmbodiedMat2,massAM)
    return EG_TM, EG_AM

In [4]:
def Manufacturing(processing_steps):
    '''
    Description:
    ------------

    This function will calculate the energy and carbon cost of manufacturing in 
    each of the manufacturing paths by multiplying the Energy demand (MJ/kg) and 
    carbon emission (kgCO2/kg) for each step by the mass that will be removed/placed in each step
    
    Variables:
    ----------
    
    processing_steps: 
    - Type: Array
                                    
    - Desc: Each of the rows in the array ius meant to represent one step in the manufacturing 
    process, for example, if you have a milling step then afinishing step, the first row will 
    relate to the costs and mass removed of the milling step and the finishing step will be described 
    on the second row - it`s done in this format - 
                                    [[Specific Energy Demand, Specific CO2 Emission, Mass],
                                     [Specific Energy Demand, Specific CO2 Emission, Mass],
                                     [Specific Energy Demand, Specific CO2 Emission, Mass]]
    
        
    Returns:
    --------
    
    result:
    - Type: Array in this format
    - Desc:
    
    
    '''
    
    
    result = np.multiply(processing_steps[:,:2],processing_steps[:,[2]])
    result = np.sum(result, axis=0)
    return result


In [5]:
# def Transport(distanceAM, distanceTM, methodAM, methodTM):
def Transport():
    '''
    Description:
    ------------

    
    
    Variables:
    ----------
    
    
    Returns:
    --------
    
    
    
    
    '''
    # https://www.section.io/engineering-education/using-geopy-to-calculate-the-distance-between-two-points/#:~:text=Geopy%20is%20a%20Python%20library,well%20as%20other%20data%20sources.
    # https://towardsdatascience.com/driving-distance-between-two-or-more-places-in-python-89779d691def
    # https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b
    return 0

In [6]:
def Use(useType, dif_mass):
    '''
    Description:
    ------------

    This function will calculate the "gain" (the decrease of environmental cost 
    due to bigger efficiecies in geometry in AM) in energy demand and carbon emissions
    when the produced piece is in use.
    
    Variables:
    ----------
    
    useType: 
    - Type: String
    - Desc: Will be used as a key to the dictionaries of energy savings and carbon demand 
    decrease per kg of decreased weight
    
    dif_mass: 
    - Type: float
    - Desc: Stores the difference in mass between the mass of the finalized product 
    manufactured through TM and AM
    
    
    Returns:
    --------
    
    result:
    - Type: Array 
    - Desc: Stores the multiplicaion of the energy and a carbon gains in an array in 
    this format- [Energy Savings, CO2 savings]
    
    
    '''
    
    # DICTIONARY - MJ PER KG OF REDUCED WEIGHT
    energySaving = {'Long-Distance Aircraft': 200*(10**3),'Short-Distance Aircraft': 150*(10**3),
                    'Articulated Truck': 260,'Passenger Car - Diesel': 210, 'Passenger Car - Gasoline': 230}
    
    # DICTIONARY - KG OF CO2 PER KG OF REDUCED WEIGHT
    CO2Saving = {'Long-Distance Aircraft': 13.6*(10**3),'Short-Distance Aircraft': 10.2*(10**3),
                    'Articulated Truck': 18.5,'Passenger Car - Diesel': 14.9, 'Passenger Car - Gasoline': 15}
    
    
    # Creating the array that will be multiplied by the difference in mass, by using the key on the dictionaries
    valuesUse = [energySaving[useType],CO2Saving[useType]]
    
    # Mutiplies the array by the difference in mass arriving at the output
    result = np.multiply(valuesUse, dif_mass)
    
    return result

In [7]:
# def Disposal(recyclability, emb_virgin, emb_waste, emb_rec, mass):
 
def Disposal():
    '''
    Description:
    ------------

    Using the "Substitution Method" the environmental impact of the disposal is calculated
    
    Variables:
    ----------
    
    recyclability: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    emb_virgin: 
    - Type: Float
    - Desc: Stores the embodied impact from virgin material input per unit of material
    
    emb_waste: 
    - Type: Float
    - Desc: Stores the embodied impact for disposal of material per unit of material
    
    emb_rec: 
    - Type: Float
    - Desc: Stores the embodied impact from recycled material input per unit of material
    
    mass: 
    - Type: Float
    - Desc: Stores the mass that is being disposed of
    
    Returns:
    --------
    
    result:
    - Type: Array
    - Desc: 
    

    
    
    '''
#     E = recyclability*emb_rec + (1-recyclability)*(emb_vir+emb_waste)
#     result = E*mass

#     return result
    return 0

In [8]:
def Calculation(imassTM, fmassTM,embodiedTMe,embodiedTMc, embodiedAMe,embodiedAMc,
                TM11,TM12,TM13,
                TM21,TM22,TM23,
                TM31,TM32,TM33,TMsteps,
                AM11,AM12,AM13,
                AM21,AM22,AM23,
                AM31,AM32,AM33,AMsteps,
                mass_reduction_perc, finishing_perc, support_perc,
                eff_energ_conv,
                CO2equivalent, recyclabilityAM, recyclabiltyTM,
                useType): 
    '''
    Description:
    ------------

    This is the core function of the program as it pertains to the calculation, 
    it controls and calls all the other functions into action. It has been separated 
    from the main function since it is called more than once, and in order to be more 
    efficient in code it was separated
    
    Variables:
    ----------
    
    imassTM: 
    - Type: float
    - Desc: Stores the initial mass of the TM workpiece
    
    
    fmassTM: 
    - Type: float
    - Desc: Stores the final mass after the workpiece has been finished in TM
    
    
    embodiedTMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in TM
    
    
    embodiedTMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in TM
    
    
    embodiedAMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in AM
    
    
    embodiedAMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in AM
    
    
    TM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in first step 
    of the TM path
    
    
    TM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of TM path
    
    
    TM13: 
    - Type: float
    - Desc: Stores the mass that will be removed in first step of the TM path
    
    
    TM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in second step 
    of the TM path
    
    
    TM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of TM path
    
    
    TM23: 
    - Type: float
    - Desc: Stores the mass that will be removed in second step of the TM path
    
    
    TM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in third step 
    of the TM path
    
    
    TM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of TM path
    
    
    TM33: 
    - Type: float
    - Desc: Stores the mass that will be removed in third step of the TM path
    
    
    TMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the TM path, as in if the number 3 is selected all 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered
    
    
    AM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in first step 
    of the AM path
    
    
    AM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of AM path
    
    
    AM13: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in first step of the AM path
    
    
    AM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in second step 
    of the AM path
    
    
    AM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of AM path
    
    
    AM23: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in second step of the AM path
    
    
    AM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in third step 
    of the AM path
    
    
    AM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of AM path
    
    
    AM33: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in third step of the AM path
    
    
    AMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the AM path, as in if the number 3 is selected all 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered
    
    
    mass_reduction_perc: 
    - Type: float
    - Desc: Percentage of the final mass of the product produced through TM, 
    that will be reduced due to geometry optimization in AM path
    
    
    finishing_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to machine allowance that will later need to be removed
    
    
    support_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to the need to create supports during the build of a particular piece
    
    
    eff_energ_conv: 
    - Type: float
    - Desc: Stores the efficiency of the Power Grid considered in the simulation
    
    
    CO2equivalent: 
    - Type: float
    - Desc: Stores the equivalent CO2 emissions due to the energy required to 
    produce the piece in the considered Power Grid in the simulation
    
    
    recyclability: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    
    useType: 
    - Type: string
    - Desc: Stores the input of where the user wants to consider thepiece to be used in;

    
    
    Returns:
    --------
    
    EnergyBarsTM:
    - Type: dictionary
    - Desc: Stores the energy impact result of each of the life cycle phases of the TM path
    
    CO2BarsTM:
    - Type: dictionary
    - Desc: Stores the CO2 impact result of each of the life cycle phases of the TM path
    
    EnergyBarsAM:
    - Type: dictionary
    - Desc: Stores the energy impact result of each of the life cycle phases of the AM path
    
    CO2BarsAM:
    - Type: dictionary
    - Desc: Stores the CO2 impact result of each of the life cycle phases of the AM path
    
    
    
    
    '''
    
    ############## MATERIAL PRODUCTION
#     print(fmassTM, fmassTM*(1-mass_reduction_perc))
    massAM = fmassTM*(1-mass_reduction_perc)/(1-(support_perc+finishing_perc))
#     print(massAM)
    
    TMmat_prod, AMmat_prod = MaterialProduction([embodiedTMe,embodiedTMc] , [embodiedAMe,embodiedAMc] , imassTM, massAM)
    
    ############# MANUFACTURING
    
    removed_massTM = imassTM-fmassTM
    
    TM_processing = np.array([[TM11,TM12,TM13*removed_massTM],[TM21,TM22,TM23*removed_massTM],[TM31,TM32,TM33*removed_massTM]])
    uTM_processing = TM_processing
    uTM_processing[(TMsteps+1):] = 0
    
    AM_processing = np.array([[AM11,AM12,AM13*massAM],[AM21,AM22,AM23*massAM],[AM31,AM32,AM33*massAM]]) 
    uAM_processing = AM_processing
    uAM_processing[(AMsteps+1):] = 0
    
    TMmanuf = Manufacturing(uTM_processing)
    AMmanuf = Manufacturing(uAM_processing)
    
    ############# TRANSPORT
    
    TMtransport = Transport()
    AMtransport = Transport()
    
    ############# USE
    
    reducedWeight = fmassTM*(-mass_reduction_perc) 
    AMuse = Use(useType, reducedWeight)
    
    ############# DISPOSAL
    
    TMdisposal = Disposal()
    AMdisposal = Disposal()
    
    EnergyBarsTM  = {'Material Production':TMmat_prod[0], 'Manufacturing':TMmanuf[0], 'Transport':TMtransport, 'Use':0, 'Disposal':TMdisposal}
    CO2BarsTM = {'Material Production':TMmat_prod[1], 'Manufacturing':TMmanuf[1], 'Transport':TMtransport, 'Use':0, 'Disposal':TMdisposal}
    
    EnergyBarsTM["Total"] = sum(EnergyBarsTM.values())
    CO2BarsTM["Total"] = sum(CO2BarsTM.values())
    
    EnergyBarsAM  = {'Material Production':AMmat_prod[0], 'Manufacturing':AMmanuf[0], 'Transport':AMtransport, 'Use':AMuse[0], 'Disposal':AMdisposal}
    CO2BarsAM = {'Material Production':AMmat_prod[1], 'Manufacturing':AMmanuf[1], 'Transport':AMtransport, 'Use':AMuse[1], 'Disposal':AMdisposal}
    
    EnergyBarsAM["Total"] = sum(EnergyBarsAM.values())
    CO2BarsAM["Total"] = sum(CO2BarsAM.values())
    
#     return (EnergyBarsTM,EnergyBarsAM)
    return (EnergyBarsTM,CO2BarsTM,EnergyBarsAM,CO2BarsAM)

In [9]:
Calculation(imassTM = 5.13, fmassTM=2.04,embodiedTMe=300,embodiedTMc=39.5, embodiedAMe=723,embodiedAMc=40,
              TM11 = 2.28,TM12 = 0.17,TM13 = 0.85,
              TM21 = 2.28,TM22 = 0.17,TM23 = 0.15,
              TM31 = 0,TM32 = 0,TM33 = 0,TMsteps=2,
              AM11 = 176.5,AM12 = 9.6,AM13 = 1,
              AM21 = 18.5,AM22 = 20.4,AM23 = 0.1,
              AM31 = 0,AM32 = 0,AM33 = 0,AMsteps=2,
              mass_reduction_perc=0, finishing_perc=0.1, support_perc = 0.2,
              eff_energ_conv = 0.4,
              CO2equivalent = 20, recyclabilityAM=0.15, recyclabiltyTM=0.15,
              useType = 'Long-Distance Aircraft')

({'Material Production': 1539.0,
  'Manufacturing': 7.045199999999999,
  'Transport': 0,
  'Use': 0,
  'Disposal': 0,
  'Total': 1546.0452},
 {'Material Production': 202.635,
  'Manufacturing': 0.5253,
  'Transport': 0,
  'Use': 0,
  'Disposal': 0,
  'Total': 203.16029999999998},
 {'Material Production': 2107.0285714285715,
  'Manufacturing': 519.7628571428572,
  'Transport': 0,
  'Use': 0.0,
  'Disposal': 0,
  'Total': 2626.7914285714287},
 {'Material Production': 116.57142857142858,
  'Manufacturing': 33.922285714285714,
  'Transport': 0,
  'Use': 0.0,
  'Disposal': 0,
  'Total': 150.4937142857143})

In [10]:
def Completed(imassTM = 5.13, fmassTM=2.04,embodiedTMe=668,embodiedTMc=39.5, embodiedAMe=723,embodiedAMc=40,
              TM11 = 2.28,TM12 = 0.17,TM13 = 0.85,
              TM21 = 18.5,TM22 = 1.53,TM23 = 0.15,
              TM31 = 0,TM32 = 0,TM33 = 0,TMsteps=2,
              AM11 = 176.5,AM12 = 9.6,AM13 = 1,
              AM21 = 18.5,AM22 = 1.39,AM23 = 0.1,
              AM31 = 0,AM32 = 0,AM33 = 0,AMsteps=2,
              mass_reduction_perc=0, finishing_perc=0.1, support_perc = 0.2,
              eff_energ_conv = 0.4,
              CO2equivalent = 20,
              useType = 'Long-Distance Aircraft', recyclabilityAM=0.15, recyclabiltyTM=0.15, solver = True, gType = "All"):
    '''
    Description:
    ------------

    This function calls the Calculation function and is called by the ipywidgets 
    library to create the control interface for the program
    
    Variables:
    ----------
    
    imassTM: 
    - Type: float
    - Desc: Stores the initial mass of the TM workpiece
    
    
    fmassTM: 
    - Type: float
    - Desc: Stores the final mass after the workpiece has been finished in TM
    
    
    embodiedTMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in TM
    
    
    embodiedTMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in TM
    
    
    embodiedAMe: 
    - Type: float
    - Desc: Stores the embodied energy impact for the material per unit of material 
    used in AM
    
    
    embodiedAMc: 
    - Type: float
    - Desc: Stores the embodied CO2 impact for the material per unit of material 
    used in AM
    
    
    TM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in first step 
    of the TM path
    
    
    TM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of TM path
    
    
    TM13: 
    - Type: float
    - Desc: Stores the mass that will be removed in first step of the TM path
    
    
    TM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in second step 
    of the TM path
    
    
    TM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of TM path
    
    
    TM23: 
    - Type: float
    - Desc: Stores the mass that will be removed in second step of the TM path
    
    
    TM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed in third step 
    of the TM path
    
    
    TM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of TM path
    
    
    TM33: 
    - Type: float
    - Desc: Stores the mass that will be removed in third step of the TM path
    
    
    TMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the TM path, as in if the number 3 is selected all 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered
    
    
    AM11: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in first step 
    of the AM path
    
    
    AM12: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in first step 
    of AM path
    
    
    AM13: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in first step of the AM path
    
    
    AM21: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in second step 
    of the AM path
    
    
    AM22: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in second step 
    of AM path
    
    
    AM23: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in second step of the AM path
    
    
    AM31: 
    - Type: float
    - Desc: Stores the energy usage per unit of material removed/placed in third step 
    of the AM path
    
    
    AM32: 
    - Type: float
    - Desc: Stores the CO2 emission per unit of material removed in third step 
    of AM path
    
    
    AM33: 
    - Type: float
    - Desc: Stores the mass that will be removed/placed in third step of the AM path
    
    
    AMsteps: 
    - Type: float
    - Desc: Stores the ammount of steps that shpuld be considerd in this 
    simulation for the AM path, as in if the number 3 is selected all 3 
    steps will be considered, if 2 is selected only the first two steps will 
    be considered
    
    
    mass_reduction_perc: 
    - Type: float
    - Desc: Percentage of the final mass of the product produced through TM, 
    that will be reduced due to geometry optimization in AM path
    
    
    finishing_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to machine allowance that will later need to be removed
    
    
    support_perc: 
    - Type: float
    - Desc: Percentage of additional mass that is placed in the AM process due 
    to the need to create supports during the build of a particular piece
    
    
    eff_energ_conv: 
    - Type: float
    - Desc: Stores the efficiency of the Power Grid considered in the simulation
    
    
    CO2equivalent: 
    - Type: float
    - Desc: Stores the equivalent CO2 emissions due to the energy required to 
    produce the piece in the considered Power Grid in the simulation
    
    
    recyclability: 
    - Type: float
    - Desc: % of recyclability (combined recovery, recycling rate and recycling mateiral yield)
    
    
    useType: 
    - Type: string
    - Desc: Stores the input of where the user wants to consider thepiece to be used in
    
    
    solver: 
    - Type: string
    - Desc: Serves as a flag from the user to indicate whether the breakeven percentage 
    of mass reduction from TM to AM should be calculated;
    
    
    gType: 
    - Type: string
    - Desc: Serves as an input from the user to decide the type of graph they want to see

    
    
    Returns:
    --------    
    
    Doesn't return anything but print out the desided plot
    
    '''
    
    
    EnergyBarsTM,CO2BarsTM,EnergyBarsAM,CO2BarsAM = Calculation(imassTM, fmassTM,embodiedTMe,embodiedTMc, embodiedAMe,embodiedAMc,
                                            TM11,TM12,TM13,
                                            TM21,TM22,TM23,
                                            TM31,TM32,TM33,TMsteps,
                                            AM11,AM12,AM13,
                                            AM21,AM22,AM23,
                                            AM31,AM32,AM33,AMsteps,
                                            mass_reduction_perc, finishing_perc, support_perc,
                                            eff_energ_conv ,
                                            CO2equivalent , recyclabilityAM, recyclabiltyTM,
                                            useType)
    
    
    
    
    mass_reduc_solver=0.1
    step = 0.1
    itera = 0
    last_result = 0
    
    while solver and (itera<20):
        EnergyBarsTMs,CO2BarsTMs,EnergyBarsAMs,CO2BarsAMs = Calculation(imassTM, fmassTM,embodiedTMe,embodiedTMc, embodiedAMe,embodiedAMc,
                                            TM11,TM12,TM13,
                                            TM21,TM22,TM23,
                                            TM31,TM32,TM33,TMsteps,
                                            AM11,AM12,AM13,
                                            AM21,AM22,AM23,
                                            AM31,AM32,AM33,AMsteps,
                                            mass_reduc_solver, finishing_perc, support_perc,
                                            eff_energ_conv,
                                            CO2equivalent , recyclabilityAM, recyclabiltyTM,
                                            useType)
        
        itera += 1
        if (EnergyBarsTMs["Total"] - EnergyBarsAMs["Total"] > 0) :
            result = -1 
        else: 
            result = +1
            
        if itera == 1:
            last_result = result
            
        if last_result != result:
            step = step/2
            
        mass_reduc_solver = mass_reduc_solver + result*step
        last_result = result
    
    print("Equilibrium of reduction in mass: ",mass_reduc_solver)
    
    if gType == "All":

        fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, sharey='row')

        namesE = list(EnergyBarsTM.keys())
        valuesE = list(EnergyBarsTM.values())

        ax1.bar(namesE,valuesE, color='red')
        ax1.set_title('Traditional Manufacturing')
        ax1.set_ylabel('Energy Consumption (MJ)')

        namesC = list(CO2BarsTM.keys())
        valuesC = list(CO2BarsTM.values())

        ax3.bar(namesC,valuesC, color='red')
        ax3.set_ylabel('CO2 production (kg)')
        
        
        namesE = list(EnergyBarsAM.keys())
        valuesE = list(EnergyBarsAM.values())

        ax2.bar(namesE,valuesE, color='blue')
        ax2.set_title('Additive Manufacturing')
        ax2.set_ylabel('Energy Consumption (MJ)')

        namesC = list(CO2BarsAM.keys())
        valuesC = list(CO2BarsAM.values())

        ax4.bar(namesC,valuesC, color='blue')
        ax4.set_ylabel('CO2 production (kg)')


    else:
        namesE = list(EnergyBarsTM.keys())
        valuesE = np.subtract(list(EnergyBarsTM.values()), list(EnergyBarsAM.values()))
        
        namesC = list(CO2BarsTM.keys())
        valuesC = np.subtract(list(CO2BarsTM.values()), list(CO2BarsAM.values()))
        
        fig, (ax1,ax2) = plt.subplots(1,2)
        ax1.bar(namesE,valuesE, color='red')
        ax1.set_title('Difference in Energy Consumption')
        ax1.set_ylabel('Delta in Energy Consumption (MJ)')
        
        ax2.bar(namesC,valuesC, color='green')
        ax2.set_title('Difference in CO2 Consumption')
        ax2.set_ylabel('Delta in CO2 production (kg)')

    
    fig.set_size_inches(18.5, 10.5)
    fig.tight_layout()
    plt.show(fig)
        

In [13]:
def ReadInput(file_name):
    inputDict = {}
    f = open(file_name,'r')
    for line in f.readlines():
#         print(line)
#         line.replace('\n','')
        words = line.split('=')
        key = words[0]
        value = words[1].replace('\n','').replace('"','').replace("'",'')
        try:
            value = float(value)
        except:
            pass
        
        inputDict[key]=value
    return inputDict

a = ReadInput("input06-02-2023--14-36-56")
print(a)

{'imassTM': 5.13, 'fmassTM': 2.04, 'embodiedTMe': 668.0, 'embodiedTMc': 39.5, 'embodiedAMe': 723.0, 'embodiedAMc': 40.0, 'TM11': 2.28, 'TM12': 0.17, 'TM13': 0.85, 'TM21': 18.5, 'TM22': 1.53, 'TM23': 0.15, 'TM31': 0.0, 'TM32': 0.0, 'TM33': 0.0, 'TMsteps': 2.0, 'AM11': 176.5, 'AM12': 9.6, 'AM13': 1.0, 'AM21': 18.5, 'AM22': 1.39, 'AM23': 0.1, 'AM31': 0.0, 'AM32': 0.0, 'AM33': 0.0, 'AMsteps': 2.0, 'mass_reduction_perc': 0.0, 'finishing_perc': 0.1, 'support_perc': 0.2, 'eff_energ_conv': 0.4, 'CO2equivalent': 20.0, 'useType': 'Articulated Truck', 'recyclabilityAM': 0.15, 'recyclabiltyTM': 0.15, 'solver': 'True', 'gType': 'All'}


In [11]:
widget = ipywidgets.interactive(Completed,solver = True, mass_reduction_perc=(0,1,0.02), useType = ['Long-Distance Aircraft','Short-Distance Aircraft',
                    'Articulated Truck','Passenger Car - Diesel', 'Passenger Car - Gasoline'], gType = ['All','Difference'])
controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))

output = widget.children[-1]
button = ipywidgets.Button(description="Click Me!")

display(VBox([controls, output,button]))

def on_button_clicked(b,children=[]):
    
    title = "input"+date.now().strftime("%d-%m-%Y--%H-%M-%S")
    f= open(title,"w+")
    for i in range(len(children)-1):
         f.write(children[i].description + "=" + str(children[i].value)+"\n")
    f.close()
    with output:
        print("Input Saved as " + title+".")

button.on_click(functools.partial(on_button_clicked, children=widget.children))

In [ ]:
def SliderWidget(minimum,maximum,size,data):
    a = ipywidgets.BoundedFloatText(
        min=minimum,
        max=maximum,
        step=size,
        disabled=False,
        value = data
    )
    return a
    


In [ ]:
a = ipywidgets.BoundedFloatText(
        min=0,
        max=20,
        step=0.1,
        disabled=False
        value = 0
    )


widget = ipywidgets.interactive(Completed,solver = True, 
                                useType = ['Long-Distance Aircraft','Short-Distance Aircraft',
                    'Articulated Truck','Passenger Car - Diesel', 'Passenger Car - Gasoline'], gType = ['All','Difference'],
            imassTM =a , fmassTM=a,
            TM11 = ipywidgets.FloatText(),TM12 = a,TM13 = (0,1,0.05),
            TM21 = a,TM22 = a,TM23 = (0,1,0.05),
            TM31 = a,TM32 = a,TM33 = (0,1,0.05),TMsteps=(1,3,1),
            AM11 = a,AM12 = a,AM13 = (0,1,0.05),
            AM21 = a,AM22 = a,AM23 = (0,1,0.05),
            AM31 = a,AM32 = a,AM33 = (0,1,0.05),AMsteps=(1,3,1),
            mass_reduction_perc=(0,1,0.02), finishing_perc=(0,1,0.05), support_perc = (0,1,0.05),
            eff_energ_conv = (0,1,0.05),
            CO2equivalent = a, recyclabilityAM=(0,1,0.05), recyclabiltyTM=(0,1,0.05))


controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))

In [ ]:
fig, (ax1,ax2) = plt.subplots(2)
EnergyBarsTM  = {'Material Production':20, 'Manufacturing':20, 'Transport':20, 'Use':20, 'Disposal':20}
names = list(EnergyBarsTM.keys())
valores = list(EnergyBarsTM.values())

ax1.bar(names,valores)
ax2.bar(names,valores)

In [ ]:
def Complete(x=0,y=0,z=0,d=0, mass_reduction_perc=0.2):
    
    EnergyBars = {'Material Production':30-x, 'Manufacturing':30-y, 'Transport':30-z, 'Use':30-d, 'Disposal':30-d}
    CO2Bars = {'Material Production':30-x, 'Manufacturing':30-y, 'Transport':30-z, 'Use':30-d, 'Disposal':30-d}
    names = list(barras.keys())
    valores = list(barras.values())
    valores.append()
    fig, (ax1,ax2) = plt.subplots
    plt.bar(nomes,valores)

In [ ]:
eff_energ_conv = 0.4 # MJ/MJ %
CO2equivalent = 20 #g/MJ

In [ ]:

##### TRADITIONAL MANUFACTURING
initialMassTM = 100 #kg
finalMassTM = 76 #kg
# Primary Production

embodied_MatTM =[10000,100] # [MJ/kg, kg/kg]

processingTM = [[10000,0.3]] #Cada linha da matriz é um dos passos, primeiro item energia e por ultimo a massa retirada (colocar em % ou em massa mesmo?)





##### ADITIVE MANUFACTURING

embodied_MatAM =[10000,100] # [MJ/kg, kg/kg]

processingAM = [[10000,1.02*finalMass],[1000,0.02]]



In [ ]:
a = ipywidgets.IntSlider()]
b = ipywidgets.IntSlider()
c = ipywidgets.IntSlider()
ui = ipywidgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = ipywidgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

In [ ]:
processing_steps = np.array([[10000,100,10],[20000,200,20]])
# result = (processing_steps[:,:2]*processing_steps[:,2][:,None])
# result = np.multiply(processing_steps[:,:2],processing_steps[:,[2]])
result = np.multiply(processing_steps,2)
# x = np.dot([[10000,100,10],[20000,200,20]],1)
# x[:,2]
result

In [ ]:
diction = {"a":10,"b":20,"c":30}
diction["total"] = sum(diction.values())
diction['x']=30300
a = np.subtract(list(diction.values()),list(diction.values()))
a